# 构建神经网络模型

In [16]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [18]:
# 继承自 nn.Module 是 PyTorch 中定义神经网络的标准方式。通过继承 nn.Module
# 我们能够利用 PyTorch 提供的很多功能，比如自动求梯度、模型保存和加载等。
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()：该层将输入的图像张量展平，从形状 (batch_size, 28, 28) 转换为 (batch_size, 28*28)。
        # 因为全连接层要求输入是一个一维的向量，而 28x28 图像需要被展平。
        self.flatten = nn.Flatten()
        # 使用 nn.Sequential 来定义神经网络的多个层。nn.Sequential 会按顺序执行其中的各层
        # 确保数据按照预期的顺序通过网络
        self.linear_relu_stack = nn.Sequential(
            # nn.Linear(28*28, 512)：输入大小为 28*28，输出大小为 512。即将每个展平的 28x28 图像送入 512 个神经元的全连接层。
            nn.Linear(28*28, 512),
            # 激活函数 ReLU（Rectified Linear Unit），它对每个输入元素执行 f(x) = max(0, x)，有效避免了梯度消失问题，并且使得神经网络具有非线性特性。
            nn.ReLU(),
            # 接着将 512 个神经元的输出再次映射到 512 个神经元的空间。
            nn.Linear(512, 512),
            # 再次应用 ReLU 激活函数。
            nn.ReLU(),
            # 最后通过一个全连接层将 512 个神经元映射到 10 个输出节点，表示每个类别的得分。这里的 10 是因为我们在 MNIST 数据集上处理的是 10 个类别（数字 0 到 9）。
            nn.Linear(512, 10),
        )


# 前向传播（Forward Propagation） 是神经网络中计算输出的过程，即数据从输入层通过隐藏层传递到输出层的过程。这个过程的目的是通过网络的结构（包括所有的权重和偏置）计算出最终的预测结果。
    def forward(self, x):
        x = self.flatten(x)
        # logits = self.linear_relu_stack(x)：接下来，通过 linear_relu_stack 层将展平后的图像数据送入神经网络，进行一系列的线性变换和 ReLU 激活操作。最终的输出是一个长度为 10 的张量，每个元素表示对应类别的得分（即 logits）。
        logits = self.linear_relu_stack(x)
        # 返回 logits 张量，通常在分类任务中，我们会将这个 logits 传入 softmax 函数来得到概率分布。
        return logits

In [19]:
model = NeuralNetwork().to(device)
# 打印神经网络的层级
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [20]:
# 生成了一个随机的 28x28 图像，X 的形状是 (1, 28, 28)，表示一个图像，大小为 28x28 像素。
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
# nn.Softmax(dim=1) 是一个 Softmax 函数，它会将 logits 转换为概率分布。Softmax 会把每个类别的得分转化为 0 到 1 之间的概率值，所有类别的概率之和为 1。
pred_probab = nn.Softmax(dim=1)(logits)
# argmax(1) 是 PyTorch 中的一个操作，用于返回沿指定维度（这里是第 1 维，即类别维度）最大值的索引。在这种情况下，它会返回概率分布中最大值的索引，表示模型预测的类别。
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


In [21]:
# 取一个由 3 张 28x28 的图片数据组成的样例数据
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [22]:
# 我们初始化 nn.Flatten(展平层) 层来将每个 2 维的 28x28 图像转换成一个包含 784 像素值的连续数组
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [23]:
# nn.Linear(线性层)是一个对输入值使用自己存储的权重 (w) 和偏差 (b) 来做线性转换的模块。
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1383, -0.2204,  0.2043, -0.3019,  0.2069,  0.5872, -0.8826, -0.3109,
         -0.1619,  0.2192,  0.2496,  0.5951,  0.4707, -0.1440, -0.1824,  0.3854,
         -0.1590, -0.5347,  0.5685, -0.2642],
        [ 0.0683, -0.0900,  0.4352, -0.2329,  0.2730,  0.5447, -0.8519,  0.0010,
         -0.2811,  0.2262,  0.2658,  0.4095,  0.3901, -0.0150,  0.1309,  0.5290,
          0.1436, -0.1344,  0.3743, -0.2606],
        [ 0.2762, -0.2786,  0.3445, -0.1645,  0.2017,  0.9403, -0.5953,  0.1692,
         -0.3013,  0.3599,  0.1232,  0.2199,  0.3321,  0.0954,  0.0150,  0.1072,
         -0.5574, -0.2680,  0.3529,  0.3580]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1383, 0.0000, 0.2043, 0.0000, 0.2069, 0.5872, 0.0000, 0.0000, 0.0000,
         0.2192, 0.2496, 0.5951, 0.4707, 0.0000, 0.0000, 0.3854, 0.0000, 0.0000,
         0.5685, 0.0000],
        [0.0683, 0.0000, 0.4352, 0.0000, 0.2730, 0.5447, 0.0000, 0.0010, 0.0000,
         0.2262, 0.2658, 0.4095, 0.3901, 0.0000, 0.13

In [25]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [26]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [27]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0041,  0.0265, -0.0005,  ...,  0.0321, -0.0215,  0.0135],
        [ 0.0202, -0.0291, -0.0023,  ..., -0.0301, -0.0057,  0.0022]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0143, -0.0254], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0003, -0.0225, -0.0089,  ..., -0.0313,  0.0280,  0.0294],
        [-0.0168, -0.0302,  0.0388,  ..., -0.0149, -0.0431, -0.0166]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 